In [28]:
import pandas as pd
import os
from functools import reduce

from nlinec import get_models_dir, get_results_dir

In [29]:
ACCURACY_FILE = 'accuracy.csv'
MNLI_ACCURACY_FILE = 'mnli_accuracy.csv'

In [30]:
models = sorted(os.listdir(get_results_dir()))
print(models)

['nlinec-AE-2', 'nlinec-B-2', 'nlinec-C-2', 'nlinec-D-1', 'nlinec-D-2', 'nlinec-D-3', 'nlinec-E-2', 'roberta-large-mnli']


In [31]:
accuracy = {}
mnli_accuracy = {}
for model in models:
    if os.path.exists(os.path.join(get_results_dir(), model, ACCURACY_FILE)):
        accuracy[model] = pd.read_csv(os.path.join(get_results_dir(), model, ACCURACY_FILE), index_col=0).rename(columns={'accuracy': model})

    if os.path.exists(os.path.join(get_results_dir(), model, MNLI_ACCURACY_FILE)):
        mnli_accuracy[model] = pd.read_csv(os.path.join(get_results_dir(), model, MNLI_ACCURACY_FILE), index_col=0).rename(columns={'accuracy': model})

In [32]:
accuracy

{'nlinec-C-2':    granularity  nlinec-C-2
 0            1    0.527657
 1            2    0.588185
 2            3    0.673861,
 'nlinec-D-1':    granularity  nlinec-D-1
 0            1    0.835219
 1            2    0.360125
 2            3    0.544365,
 'nlinec-D-2':    granularity  nlinec-D-2
 0            1    0.423691
 1            2    0.588753
 2            3    0.599520,
 'nlinec-D-3':    granularity  nlinec-D-3
 0            1    0.380962
 1            2    0.499290
 2            3    0.642686,
 'nlinec-E-2':    granularity  nlinec-E-2
 0            1    0.235643
 1            2    0.046862
 2            3    0.136691,
 'roberta-large-mnli':    granularity  roberta-large-mnli
 0            1            0.460074
 1            2            0.346492
 2            3            0.441247}

In [33]:
# Merge all accuracy dataframes on the 'granularity' column
accuracy_df = reduce(lambda left, right: pd.merge(left, right, on='granularity'), accuracy.values())
accuracy_df

,granularity,nlinec-C-2,nlinec-D-1,nlinec-D-2,nlinec-D-3,nlinec-E-2,roberta-large-mnli
0,1,0.527657,0.835219,0.423691,0.380962,0.235643,0.460074
1,2,0.588185,0.360125,0.588753,0.499290,0.046862,0.346492
2,3,0.673861,0.544365,0.599520,0.642686,0.136691,0.441247


In [34]:
mnli_accuracy_df = pd.concat(mnli_accuracy.values(), axis=1)
mnli_accuracy_df

,nlinec-C-2,nlinec-D-1,nlinec-D-2,nlinec-D-3,nlinec-E-2,roberta-large-mnli
accuracy,0.74488,0.841875,0.823128,0.805502,0.700662,0.902598
